In [84]:
import itertools
import numpy as np

# Матрицы вероятностей переходов P[action][state_from][state_to]
transition_probabilities = {
    0: [[0.3, 0.5, 0.2],
        [0.2, 0.6, 0.2],
        [0.1, 0.2, 0.7]],
    1: [[0.2, 0.7, 0.1],
        [0.1, 0.4, 0.5],
        [0.1, 0.2, 0.7]],
    2: [[0.3, 0.4, 0.3],
        [0.2, 0.6, 0.2],
        [0.1, 0.3, 0.6]],
}

# Матрицы вознаграждений R[action][state_from][state_to]
rewards = {
    0: [[110, 100, 70],
        [100,  80, 50],
        [ 80,  60, 40]],
    1: [[120, 100, 70],
        [110, 100, 90],
        [100,  70, 60]],
    2: [[110,  80, 50],
        [100,  60, 40],
        [ 80,  70, 60]],
}

actions = [0, 1, 2]  # Доступные действия (индексы)
states = [0, 1, 2]   # Индексы состояний: 0 - «Отличный», 1 - «Хороший», 2 - «Удовлетворительный»
months = 3  # Количество месяцев (длительность стратегии)
state_names = ["Отличный", "Хороший", "Удовлетворительный"]  # Человекочитаемые имена состояний

def calculate_expected_reward(strategy, initial_state):
    """
    Вычисление ожидаемого вознаграждения для заданной стратегии действий.
    
    Параметры:
    strategy (list[int]): Стратегия действий (порядок действий на каждый месяц).
    initial_state (int): Начальное состояние системы.
    
    Возвращает:
    float: Ожидаемое вознаграждение для стратегии.
    """
    # Инициализация вероятностей состояний: на старте мы находимся в начальном состоянии
    state_probabilities = [0.0, 0.0, 0.0]
    state_probabilities[initial_state] = 1.0
    
    total_reward = 0.0  # Общее ожидаемое вознаграждение
    
    # Процесс для каждого действия в стратегии
    for action in strategy:
        new_state_probabilities = [0.0, 0.0, 0.0]  # Вероятности для нового состояния
        # Проходим по всем возможным переходам между состояниями
        for i in range(3):
            for j in range(3):
                transition_probability = transition_probabilities[action][i][j]
                reward = rewards[action][i][j]
                total_reward += state_probabilities[i] * transition_probability * reward
                new_state_probabilities[j] += state_probabilities[i] * transition_probability
        
        # Обновляем вероятности состояний для следующего шага
        state_probabilities = new_state_probabilities
    
    return total_reward

# Поиск наилучшей стратегии для каждого начального состояния
for initial_state in range(3):
    best_strategy = None
    best_reward = -1e9  # Начальная очень низкая оценка

    # Генерация всех возможных стратегий на протяжении заданного количества месяцев
    for strategy in itertools.product(actions, repeat=months):
        expected_reward = calculate_expected_reward(strategy, initial_state)
        # Обновляем лучшую стратегию, если текущая дает большее вознаграждение
        if expected_reward > best_reward:
            best_strategy = strategy
            best_reward = expected_reward
    
    # Выводим результаты для каждого начального состояния
    print(f"Для начального состояния «{state_names[initial_state]}»:")
    print(f"  Лучшая стратегия действий: {best_strategy}")
    print(f"  Ожидаемое вознаграждение: {best_reward:.2f}\n")


Для начального состояния «Отличный»:
  Лучшая стратегия действий: (1, 1, 1)
  Ожидаемое вознаграждение: 278.40

Для начального состояния «Хороший»:
  Лучшая стратегия действий: (1, 1, 1)
  Ожидаемое вознаграждение: 257.25

Для начального состояния «Удовлетворительный»:
  Лучшая стратегия действий: (2, 1, 1)
  Ожидаемое вознаграждение: 222.65



___

In [87]:
import itertools
import numpy as np

# Матрицы переходов и доходов для каждого действия
transition_matrix = {
    0: [[0.3, 0.5, 0.2],
        [0.2, 0.6, 0.2],
        [0.1, 0.2, 0.7]],
    1: [[0.2, 0.7, 0.1],
        [0.1, 0.4, 0.5],
        [0.1, 0.2, 0.7]],
    2: [[0.3, 0.4, 0.3],
        [0.2, 0.6, 0.2],
        [0.1, 0.3, 0.6]],
}

reward_matrix = {
    0: [[110, 100, 70],
        [100,  80, 50],
        [ 80,  60, 40]],
    1: [[120, 100, 70],
        [110, 100, 90],
        [100,  70, 60]],
    2: [[110,  80, 50],
        [100,  60, 40],
        [ 80,  70, 60]],
}

actions = [0, 1, 2]  # Индексы доступных действий
states = [0, 1, 2]   # Состояния: 0 - «Отличный», 1 - «Хороший», 2 - «Удовлетворительный»

def evaluate_policy(policy):
    """
    Оценка политики: вычисление среднего дохода и стационарного распределения состояний.
    
    Параметры:
    policy (list[int]): Стратегия действий для каждого состояния.
    
    Возвращает:
    float: Средний доход от применения политики.
    numpy.ndarray: Стационарное распределение состояний.
    """
    # Инициализация матрицы переходов и вектора вознаграждений для данной политики
    transition_matrix_policy = np.zeros((3, 3))
    reward_vector_policy = np.zeros(3)
    
    # Для каждого состояния, вычисляем ожидаемое вознаграждение и вероятности перехода
    for state in states:
        action = policy[state]
        transition_matrix_policy[state, :] = transition_matrix[action][state]
        for next_state in states:
            reward_vector_policy[state] += transition_matrix[action][state][next_state] * reward_matrix[action][state][next_state]

    # Решение задачи с собственными значениями для нахождения стационарного распределения
    eigenvalues, eigenvectors = np.linalg.eig(transition_matrix_policy.T)
    stationary_state_idx = np.argmin(np.abs(eigenvalues - 1.0))
    stationary_distribution = np.real(eigenvectors[:, stationary_state_idx])
    
    # Нормировка стационарного распределения
    stationary_distribution /= stationary_distribution.sum()

    # Рассчитываем средний доход
    average_reward = float(np.dot(stationary_distribution, reward_vector_policy))
    return average_reward, stationary_distribution

# Инициализация переменных для поиска лучшей политики
best_policy = None
best_gain = -1e9  # Начальное значение для максимального дохода
best_stationary_distribution = None

# Перебор всех возможных политик и выбор лучшей
for policy in itertools.product(actions, repeat=3):
    gain, stationary_distribution = evaluate_policy(policy)
    if gain > best_gain:
        best_gain = gain
        best_policy = policy
        best_stationary_distribution = stationary_distribution

# Вывод результатов
print("Лучшая стационарная стратегия для состояний (0, 1, 2):", best_policy)
print("Стационарное распределение состояний μ:", np.round(best_stationary_distribution, 3))
print("Средний доход g при лучшей стратегии:", round(best_gain, 2))


Лучшая стационарная стратегия для состояний (0, 1, 2): (1, 1, 2)
Стационарное распределение состояний μ: [0.111 0.383 0.506]
Средний доход g при лучшей стратегии: 80.86


___

In [89]:
import numpy as np

# 1) Задаём P и R по условию задачи
P = {
    0: [[0.3, 0.5, 0.2],
        [0.2, 0.6, 0.2],
        [0.1, 0.2, 0.7]],
    1: [[0.2, 0.7, 0.1],
        [0.1, 0.4, 0.5],
        [0.1, 0.2, 0.7]],
    2: [[0.3, 0.4, 0.3],
        [0.2, 0.6, 0.2],
        [0.1, 0.3, 0.6]],
}

R = {
    0: [[110, 100, 70],
        [100,  80, 50],
        [ 80,  60, 40]],
    1: [[120, 100, 70],
        [110, 100, 90],
        [100,  70, 60]],
    2: [[110,  80, 50],
        [100,  60, 40],
        [ 80,  70, 60]],
}

num_states = 3
actions = [0, 1, 2]  # 0=скидка, 1=доставка, 2=ничего
state_names = ["Отличный", "Хороший", "Удовлетворительный"]
action_names = ["3% скидка", "Бесплатная доставка", "Ничего"]

def evaluate_policy(policy):
    """Policy evaluation (gain g и bias h)"""
    m = num_states
    # 1) Собираем P_pi и r_pi
    P_pi = np.zeros((m, m))
    r_pi = np.zeros(m)
    for i in range(m):
        a = policy[i]
        for j in range(m):
            P_pi[i, j] = P[a][i][j]
            r_pi[i] += P[a][i][j] * R[a][i][j]

    # 2) Строим и решаем систему (m+1)x(m+1) на [h0..h_{m-1}, g]
    A = np.zeros((m + 1, m + 1))
    b = np.zeros(m + 1)
    for i in range(m):
        A[i, i] = 1.0
        A[i, :m] -= P_pi[i, :]
        A[i, m] = 1.0
        b[i] = r_pi[i]
    # фиксация h[m-1] = 0
    A[m, m - 1] = 1.0
    b[m] = 0.0

    x = np.linalg.solve(A, b)
    h = x[:m]
    g = x[m]
    return g, h

def improve_policy(policy, h):
    """Policy improvement"""
    m = num_states
    new_pol = policy.copy()
    for i in range(m):
        best_q, best_a = -1e9, None
        for a in actions:
            # считаем Q(i,a) = r(i,a) + sum_j P[a][i][j]·h[j]
            q = 0.0
            for j in range(m):
                q += P[a][i][j] * R[a][i][j]  # r(i,a)
                q += P[a][i][j] * h[j]  # влияние bias
            if q > best_q:
                best_q, best_a = q, a
        new_pol[i] = best_a
    return new_pol

def policy_iteration():
    policy = [0] * num_states  # стартуем, например, всегда со "скидки"
    iteration = 0
    while True:
        #  -- вывод текущей политики --
        print(f"\nИтерация {iteration}. Текущая политика:")
        for i, a in enumerate(policy):
            print(f"  {i} -> {a}")

        # Оценка этой политики
        g, h = evaluate_policy(policy)
        print(f"  Оценка: средний доход g = {g:.4f}")

        # Улучшаем политику
        new_pol = improve_policy(policy, h)

        # Если не изменилось — готово
        if new_pol == policy:
            print("\nПолитика не изменилась. Алгоритм завершён.")
            break

        policy = new_pol
        iteration += 1

    return policy, g, h

if __name__ == "__main__":
    opt_policy, opt_gain, opt_h = policy_iteration()
    print("\n=== Итоговая оптимальная политика ===")
    for i, a in enumerate(opt_policy):
        print(f"{i} -> {a}")
    print(f"Оптимальный средний доход g* = {opt_gain:.4f}")


Итерация 0. Текущая политика:
  0 -> 0
  1 -> 0
  2 -> 0
  Оценка: средний доход g = 69.3778

Итерация 1. Текущая политика:
  0 -> 1
  1 -> 0
  2 -> 2
  Оценка: средний доход g = 77.9322

Итерация 2. Текущая политика:
  0 -> 1
  1 -> 1
  2 -> 2
  Оценка: средний доход g = 80.8642

Политика не изменилась. Алгоритм завершён.

=== Итоговая оптимальная политика ===
0 -> 1
1 -> 1
2 -> 2
Оптимальный средний доход g* = 80.8642


___

In [92]:
import numpy as np

# 1) Задаём P и R по условию
P = {
    0: [[0.3, 0.5, 0.2],
        [0.2, 0.6, 0.2],
        [0.1, 0.2, 0.7]],
    1: [[0.2, 0.7, 0.1],
        [0.1, 0.4, 0.5],
        [0.1, 0.2, 0.7]],
    2: [[0.3, 0.4, 0.3],
        [0.2, 0.6, 0.2],
        [0.1, 0.3, 0.6]],
}

R = {
    0: [[110, 100, 70],
        [100,  80, 50],
        [ 80,  60, 40]],
    1: [[120, 100, 70],
        [110, 100, 90],
        [100,  70, 60]],
    2: [[110,  80, 50],
        [100,  60, 40],
        [ 80,  70, 60]],
}

num_states = 3
actions = [0, 1, 2]  # 0=скидка, 1=доставка, 2=ничего
state_names = ["Отл.", "Хор.", "Уд."]
action_names = ["3% скидка", "доставка", "ничего"]

γ = 0.9  # коэффициент дисконтирования

def evaluate_policy_discount(policy, gamma=γ):
    """
    Решаем (I - γ Pπ) V = rπ
    возвращаем вектор V размера m.
    """
    m = num_states
    Pπ = np.zeros((m, m))
    rπ = np.zeros(m)
    for i in range(m):
        a = policy[i]
        for j in range(m):
            Pπ[i, j] = P[a][i][j]
            rπ[i] += P[a][i][j] * R[a][i][j]
    # матрица I - γ·Pπ
    A = np.eye(m) - gamma * Pπ
    V = np.linalg.solve(A, rπ)
    return V

def improve_policy_discount(policy, V, gamma=γ):
    """
    Для каждого состояния i находим a, максимизирующее
       Q(i,a) = r(i,a) + γ ∑_j P[a][i][j]·V[j]
    """
    m = num_states
    new_pol = policy.copy()
    for i in range(m):
        best_q, best_a = -1e9, None
        for a in actions:
            # r(i,a) + γ P[a][i]·V
            q = 0.0
            # мгновенное ожидание r(i,a)
            for j in range(m):
                q += P[a][i][j] * R[a][i][j]
            # плюс дисконтированное будущее
            for j in range(m):
                q += gamma * P[a][i][j] * V[j]
            if q > best_q:
                best_q, best_a = q, a
        new_pol[i] = best_a
    return new_pol

def policy_iteration_discount():
    # стартуем, например, всегда «3% скидка»
    policy = [0] * num_states
    it = 0
    while True:
        print(f"\n-- Итерация {it}, текущая политика:")
        for i, a in enumerate(policy):
            print(f"  {i} -> {a}")
        # оценка
        V = evaluate_policy_discount(policy)
        print("   V =", np.round(V, 3))
        # улучшение
        new_pol = improve_policy_discount(policy, V)
        if new_pol == policy:
            print("\nПолитика не изменилась, алгоритм завершён.")
            break
        policy = new_pol
        it += 1
    return policy, V

if __name__ == "__main__":
    opt_pol, opt_V = policy_iteration_discount()
    print("\n=== Оптимальная политика ===")
    for i, a in enumerate(opt_pol):
        print(f"{i} -> {a}")
    print("Стоимость состояний V* =", np.round(opt_V, 3))


-- Итерация 0, текущая политика:
  0 -> 0
  1 -> 0
  2 -> 0
   V = [734.13  713.251 655.289]

-- Итерация 1, текущая политика:
  0 -> 1
  1 -> 1
  2 -> 2
   V = [842.748 823.777 789.711]

Политика не изменилась, алгоритм завершён.

=== Оптимальная политика ===
0 -> 1
1 -> 1
2 -> 2
Стоимость состояний V* = [842.748 823.777 789.711]


___

In [94]:
import numpy as np
from scipy.optimize import linprog

# 1. Определяем матрицы вероятностей P и вознаграждений R по условию задачи

# P[a][s][s'] - вероятность перехода из состояния s в s' при действии a
P = {
    0: [[0.3, 0.5, 0.2],
        [0.2, 0.6, 0.2],
        [0.1, 0.2, 0.7]],
    1: [[0.2, 0.7, 0.1],
        [0.1, 0.4, 0.5],
        [0.1, 0.2, 0.7]],
    2: [[0.3, 0.4, 0.3],
        [0.2, 0.6, 0.2],
        [0.1, 0.3, 0.6]],
}

# R[a][s][s'] - вознаграждение за переход из состояния s в s' при действии a
R = {
    0: [[110, 100, 70],
        [100,  80, 50],
        [ 80,  60, 40]],
    1: [[120, 100, 70],
        [110, 100, 90],
        [100,  70, 60]],
    2: [[110,  80, 50],
        [100,  60, 40],
        [ 80,  70, 60]],
}

# Состояния и действия
state_names = ["Отличный", "Хороший", "Удовлетворительный"]
action_names = ["3% скидка", "Бесплатная доставка", "Ничего"]

# Параметры задачи
num_states = 3  # Количество состояний
num_actions = 3  # Количество действий
num_variables = num_states * num_actions  # Общее количество переменных (9)

# 2. Формируем вектор вознаграждений r для каждого состояния и действия
reward_vector = np.zeros(num_variables)
for state in range(num_states):
    for action in range(num_actions):
        idx = state * num_actions + action
        # Ожидаемое вознаграждение для пары (состояние, действие)
        reward_vector[idx] = sum(P[action][state][next_state] * R[action][state][next_state] for next_state in range(num_states))

# 3. Формируем матрицу A_eq и вектор b_eq для решения задачи линейного программирования

# A_eq и b_eq для условия балансировки потоков и нормировки
A_eq = np.zeros((num_states + 1, num_variables))
b_eq = np.zeros(num_states + 1)

# а) Баланс потоков: для каждого состояния j, учитываем все действия a
for next_state in range(num_states):
    for state in range(num_states):
        for action in range(num_actions):
            idx = state * num_actions + action
            if state == next_state:
                A_eq[next_state, idx] += 1.0
            A_eq[next_state, idx] -= P[action][state][next_state]

# б) Нормировка: сумма всех переменных x_{s,a} равна 1
A_eq[num_states, :] = 1.0
b_eq[num_states] = 1.0

# 4. Решаем задачу линейного программирования для максимизации r^T * x
# Задача сводится к минимизации -r^T * x
result = linprog(
    c=-reward_vector,  # Минмизируем -r^T * x
    A_eq=A_eq,
    b_eq=b_eq,
    bounds=[(0, None)] * num_variables,
    method='highs'  # Используем метод 'highs', возможен также 'revised simplex'
)

# Проверка успешности решения задачи
if not result.success:
    raise RuntimeError(f"Ошибка в решении LP задачи: {result.message}")

# Оптимальное решение
optimal_x = result.x
optimal_reward = reward_vector.dot(optimal_x)

# 5. Восстанавливаем политику для каждого состояния: выбираем действие с максимальной вероятностью
optimal_policy = []
for state in range(num_states):
    values = [optimal_x[state * num_actions + action] for action in range(num_actions)]
    best_action = int(np.argmax(values))  # Действие с максимальной вероятностью
    optimal_policy.append(best_action)

# 6. Выводим результаты
print(f"Оптимальное среднее вознаграждение g* = {optimal_reward:.4f}\n")
print("Оптимальная стационарная детерминированная политика:")
for state in range(num_states):
    print(f"  {state} → {optimal_policy[state]}")


Оптимальное среднее вознаграждение g* = 80.8642

Оптимальная стационарная детерминированная политика:
  0 → 1
  1 → 1
  2 → 2


___

In [95]:
import numpy as np
from scipy.optimize import linprog

# 1. Данные задачи

# P[a][s][s'] - вероятность перехода из состояния s в состояние s' при действии a
transition_probabilities = {
    0: [[0.3, 0.5, 0.2],
        [0.2, 0.6, 0.2],
        [0.1, 0.2, 0.7]],
    1: [[0.2, 0.7, 0.1],
        [0.1, 0.4, 0.5],
        [0.1, 0.2, 0.7]],
    2: [[0.3, 0.4, 0.3],
        [0.2, 0.6, 0.2],
        [0.1, 0.3, 0.6]],
}

# R[a][s][s'] - вознаграждение за переход из состояния s в s' при действии a
rewards = {
    0: [[110, 100, 70],
        [100,  80, 50],
        [ 80,  60, 40]],
    1: [[120, 100, 70],
        [110, 100, 90],
        [100,  70, 60]],
    2: [[110,  80, 50],
        [100,  60, 40],
        [ 80,  70, 60]],
}

# Количество состояний и действий
num_states = 3  # Количество состояний
num_actions = 3  # Количество действий
discount_factor = 0.9  # Дисконт-фактор

# Начальное распределение: стартуем из состояния "Отличный"
initial_distribution = np.array([1.0, 0.0, 0.0])

# 2. Формируем вектор вознаграждений r размером num_states * num_actions
reward_vector = np.zeros(num_states * num_actions)
for state in range(num_states):
    for action in range(num_actions):
        idx = state * num_actions + action
        # Ожидаемое вознаграждение для состояния и действия (s, a)
        reward_vector[idx] = sum(transition_probabilities[action][state][next_state] * rewards[action][state][next_state] 
                                 for next_state in range(num_states))

# 3. Формируем матрицу равенств A_eq и вектор b_eq для линейного программирования

# A_eq - матрица коэффициентов, b_eq - вектор правых частей
A_eq = np.zeros((num_states, num_states * num_actions))
b_eq = initial_distribution.copy()

# Формируем систему уравнений для каждого состояния
for next_state in range(num_states):
    for state in range(num_states):
        for action in range(num_actions):
            idx = state * num_actions + action
            # Добавляем коэффициент для y[j, a] на левой части
            if state == next_state:
                A_eq[next_state, idx] += 1.0
            # Добавляем коэффициент для -γ·P[a][s][j]·y[s, a] на правой части
            A_eq[next_state, idx] -= discount_factor * transition_probabilities[action][state][next_state]

# 4. Решаем задачу линейного программирования (LP): maximize r^T * y <=> minimize -r^T * y
result = linprog(
    c=-reward_vector,  # Минмизируем -r^T * y
    A_eq=A_eq,
    b_eq=b_eq,
    bounds=[(0, None)] * (num_states * num_actions),
    method='highs'  # Используем метод 'highs', можно также использовать 'revised simplex'
)

# Проверяем успешность решения задачи
if not result.success:
    raise RuntimeError(f"Ошибка в решении LP задачи: {result.message}")

# Оптимальные значения переменных y
optimal_y = result.x
optimal_value = reward_vector.dot(optimal_y)  # Оптимальный дисконтированный доход

# 5. Восстанавливаем стратегию (политику): для каждого состояния выбираем действие с максимальным y
optimal_policy = []
for state in range(num_states):
    action_values = [optimal_y[state * num_actions + action] for action in range(num_actions)]
    optimal_action = int(np.argmax(action_values))  # Действие с максимальной вероятностью
    optimal_policy.append(optimal_action)

# 6. Выводим результаты
state_labels = ["Отличный", "Хороший", "Удовлетворительный"]
action_labels = ["3% скидка", "Бесплатная доставка", "Ничего"]

# Вывод оптимального дисконтированного дохода
print(f"Оптимальный дисконтированный доход V* = {optimal_value:.4f}\n")

# Вывод оптимальной стратегии для каждого состояния
print("Оптимальная стратегия (политика):")
for state, action in enumerate(optimal_policy):
    print(f"{state} → {action}")


Оптимальный дисконтированный доход V* = 842.7485

Оптимальная стратегия (политика):
0 → 1
1 → 1
2 → 2


___


In [96]:
import numpy as np
import pulp

# 1) Данные MDP
P = {
    0: [[0.3, 0.5, 0.2],
        [0.2, 0.6, 0.2],
        [0.1, 0.2, 0.7]],
    1: [[0.2, 0.7, 0.1],
        [0.1, 0.4, 0.5],
        [0.1, 0.2, 0.7]],
    2: [[0.3, 0.4, 0.3],
        [0.2, 0.6, 0.2],
        [0.1, 0.3, 0.6]],
}
R = {
    0: [[110, 100, 70],
        [100,  80, 50],
        [ 80,  60, 40]],
    1: [[120, 100, 70],
        [110, 100, 90],
        [100,  70, 60]],
    2: [[110,  80, 50],
        [100,  60, 40],
        [ 80,  70, 60]],
}

m = 3  # состояний
A = 3  # действий
gamma = 0.9  # дисконт
d0 = [1.0, 0.0, 0.0]  # стартуем из «Отличного»

# 2) Предсчитаем «моментный» r[s,a]
r = {}
for s in range(m):
    for a in range(A):
        r[s, a] = sum(P[a][s][j] * R[a][s][j] for j in range(m))

# 3) Формируем LP через PuLP
model = pulp.LpProblem("Discounted_MDP", pulp.LpMaximize)

# переменные y[s,a] >= 0
y = {(s, a): pulp.LpVariable(f"y_{s}_{a}", lowBound=0)
     for s in range(m) for a in range(A)}

# --- целевая функция: max sum r[s,a]*y[s,a]
model += pulp.lpSum(r[s, a] * y[s, a] for s in range(m) for a in range(A)), "Obj"

# --- баланс: для каждого j: sum_a y[j,a] - γ sum_{s,a} P[a][s][j]*y[s,a] == d0[j]
for j in range(m):
    expr = pulp.lpSum(y[j, a] for a in range(A)) \
           - gamma * pulp.lpSum(P[a][s][j] * y[s, a]
                               for s in range(m) for a in range(A))
    model += (expr == d0[j], f"flow_state_{j}")

# --- решаем
model.solve(pulp.PULP_CBC_CMD(msg=False))

# 4) Собираем результаты
print("=== Результат LP ===")
print("Status:", pulp.LpStatus[model.status])
print("Оптимальный V* = ", pulp.value(model.objective))
print()

# оптимальная политика
print("Оптимальная политика:")
for s in range(m):
    best_a, best_val = None, -1e9
    for a in range(A):
        val = y[s, a].value()
        if val > best_val:
            best_val, best_a = val, a
    print(f"  Состояние {s} → действие {best_a}  (y={best_val:.4f})")
print()

# 5) Анализ чувствительности
print("=== Dual variables (shadow prices) для уравнений потока ===")
for cname, con in model.constraints.items():
    # .pi — дуальная переменная (shadow price)
    print(f"  {cname:15s}  π = {con.pi: .4f}")

print()
print("=== Reduced costs для y[s,a] ===")
for (s, a), var in y.items():
    # var.dj — reduced cost
    print(f"  y[{s},{a}] = {var.value():.4f}    reduced cost = {var.dj: .4f}")

=== Результат LP ===
Status: Optimal
Оптимальный V* =  842.7484618000001

Оптимальная политика:
  Состояние 0 → действие 1  (y=2.0879)
  Состояние 1 → действие 1  (y=3.7930)
  Состояние 2 → действие 2  (y=4.1191)

=== Dual variables (shadow prices) для уравнений потока ===
  flow_state_0     π =  842.7485
  flow_state_1     π =  823.7773
  flow_state_2     π =  789.7114

=== Reduced costs для y[s,a] ===
  y[0,0] = 0.0000    reduced cost = -5.3585
  y[0,1] = 2.0879    reduced cost =  0.0000
  y[0,2] = 0.0000    reduced cost = -25.4245
  y[1,0] = 0.0000    reduced cost = -7.0948
  y[1,1] = 3.7930    reduced cost = -0.0000
  y[1,2] = 0.0000    reduced cost = -21.0948
  y[2,0] = 0.0000    reduced cost = -20.0659
  y[2,1] = 0.0000    reduced cost = -2.0659
  y[2,2] = 4.1191    reduced cost = -0.0000


In [97]:
import numpy as np

# policy_iteration_discount взяли из предыдущего примера, 
# но сделали функцию, принимающую gamma
def policy_iteration_discount(P, R, gamma):
    m = len(P[0])
    actions = list(P.keys())
    policy = [actions[0]]*m
    while True:
        # evaluate
        Pπ = np.zeros((m,m))
        rπ = np.zeros(m)
        for i in range(m):
            a = policy[i]
            for j in range(m):
                Pπ[i,j] = P[a][i][j]
                rπ[i]  += P[a][i][j] * R[a][i][j]
        V = np.linalg.solve(np.eye(m) - gamma*Pπ, rπ)
        # improve
        new_pol = policy.copy()
        for i in range(m):
            qs = []
            for a in actions:
                q = sum(P[a][i][j]*R[a][i][j] for j in range(m)) \
                    + gamma*sum(P[a][i][j]*V[j] for j in range(m))
                qs.append(q)
            new_pol[i] = actions[int(np.argmax(qs))]
        if new_pol == policy:
            break
        policy = new_pol
    return policy

# Матрицы P и R — как раньше
P = {
    0: [[0.3,0.5,0.2],[0.2,0.6,0.2],[0.1,0.2,0.7]],
    1: [[0.2,0.7,0.1],[0.1,0.4,0.5],[0.1,0.2,0.7]],
    2: [[0.3,0.4,0.3],[0.2,0.6,0.2],[0.1,0.3,0.6]],
}
R = {
    0: [[110,100,70],[100,80,50],[80,60,40]],
    1: [[120,100,70],[110,100,90],[100,70,60]],
    2: [[110,80,50],[100,60,40],[80,70,60]],
}

# Сканируем γ и фиксируем, какая политика в итоге получилась
gammas = np.linspace(0.1,0.99,19)
policies = []
for γ in gammas:
    pol = tuple(policy_iteration_discount(P, R, γ))
    policies.append(pol)

# Выводим интервалы γ, на которых политика стабильна
intervals = []
start = gammas[0]
prev = policies[0]
for g, pol in zip(gammas[1:], policies[1:]):
    if pol != prev:
        intervals.append((start, prev, g))
        start = g
        prev = pol
# последний кусок
intervals.append((start, prev, gammas[-1]))

print("Промежутки дисконтирования и политики:")
for low, pol, high in intervals:
    print(f"γ ∈ [{low:.2f}, {high:.2f}] → политика {pol}")

Промежутки дисконтирования и политики:
γ ∈ [0.10, 0.35] → политика (1, 1, 1)
γ ∈ [0.35, 0.99] → политика (1, 1, 2)
